In [1]:
!pip install python-terrier
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 6.8 MB/s 
     |████████████████████████████████| 1.1 MB 55.9 MB/s 
     |████████████████████████████████| 69 kB 8.8 MB/s 
     |████████████████████████████████| 303 kB 58.3 MB/s 
     |████████████████████████████████| 46 kB 5.0 MB/s 
     |████████████████████████████████| 45 kB 3.8 MB/s 
     |████████████████████████████████| 72 kB 1.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 294 kB 67.9 MB/s 
     |████████████████████████████████| 126 kB 71.7 MB/s 
     |████████████████████████████████| 1.2 MB 46.3 MB/s 
     |████████████████████████████████| 285 kB 69.3 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.8.1-py3-none-any.whl size=104091 sha256=6f57a67c21988abaf1c1a5c168432f4cdc18c49fa2cc5

In [2]:
!wget https://files.webis.de/data-in-progress/image_text_ocr.jsonl
!wget https://zenodo.org/record/6786949/files/touche-task3-001-050-relevance.qrels?download=1 -O task3-qrels.txt
!wget https://zenodo.org/record/6786949/files/topics.xml?download=1 -O task3-topics.xml

--2022-09-28 18:31:51--  https://files.webis.de/data-in-progress/image_text_ocr.jsonl
Resolving files.webis.de (files.webis.de)... 141.54.132.200
Connecting to files.webis.de (files.webis.de)|141.54.132.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 294746702 (281M) [application/octet-stream]
Saving to: ‘image_text_ocr.jsonl’

image_text_ocr.json 100%[===================>] 281.09M  12.1MB/s    in 37s     

2022-09-28 18:32:29 (7.57 MB/s) - ‘image_text_ocr.jsonl’ saved [294746702/294746702]

--2022-09-28 18:32:29--  https://zenodo.org/record/6786949/files/touche-task3-001-050-relevance.qrels?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 558028 (545K) [application/octet-stream]
Saving to: ‘task3-qrels.txt’

task3-qrels.txt     100%[===================>] 544.95K   721KB/s    in 0.8s    

2022-09-28 18:32:32 (721 KB

The next step is to initialise PyTerrier. This is performed using PyTerrier's `init()` method. The `init()` method is needed as PyTerrier must download Terrier's jar file and start the Java virtual machine. We prevent `init()` from being called more than once by checking `started()`.

In [4]:
import pyterrier as pt
if not pt.started():
  pt.init()

### Documents, Indexing and Indexes

Much of PyTerrier's view of the world is wrapped up in Pandas dataframes. Let's consider some textual documents in a dataframe.


In [5]:
# we need to import pandas. We commonly rename it to pd, to make commands shorter
import pandas as pd

# lets not truncate output too much
pd.set_option('display.max_colwidth', 150)

docs_df = pd.DataFrame([
    {'qid': '1', 'query': 'Argument Against Teachers should get Tenure', 'docno': 'd1', 'body': 'Teachers should not get tenure because they should stay motivated'},
    {'qid': '1', 'query': 'Argument Against Teachers should get Tenure', 'docno': 'd2', 'body': 'Teachers should get tenure to keep them motivated'},
    {'qid': '2', 'query': 'Argument Pro Teachers should get Tenure', 'docno': 'd1', 'body': 'Teachers should not get tenure because they should stay motivated'},
    {'qid': '2', 'query': 'Argument Pro for Teachers should get Tenure', 'docno': 'd2', 'body': 'Teachers should get tenure to keep them motivated'},


    {'qid': '3', 'query': 'Argument Against Should body cameras be mandatory for police', 'docno': 'd1', 'body': 'This meme shows that the police should get body cameras'},
    {'qid': '3', 'query': 'Argument Against Should body cameras be mandatory for police', 'docno': 'd2', 'body': 'This meme shows that the police is trustworthy, so no body camerass required.'},
    {'qid': '4', 'query': 'Argument Pro Should body cameras be mandatory for police', 'docno': 'd1', 'body': 'This meme shows that the police should get body cameras'},
    {'qid': '4', 'query': 'Argument Pro Should body cameras be mandatory for police', 'docno': 'd2', 'body': 'This meme shows that the police is trustworthy, so no body camerass required.'},
    ])

docs_df['text'] = docs_df['body']
del docs_df['body']
docs_df

,qid,query,docno,text
0,1,Argument Against Teachers should get Tenure,d1,Teachers should not get tenure because they should stay motivated
1,1,Argument Against Teachers should get Tenure,d2,Teachers should get tenure to keep them motivated
2,2,Argument Pro Teachers should get Tenure,d1,Teachers should not get tenure because they should stay motivated
3,2,Argument Pro for Teachers should get Tenure,d2,Teachers should get tenure to keep them motivated
4,3,Argument Against Should body cameras be mandatory for police,d1,This meme shows that the police should get body cameras
5,3,Argument Against Should body cameras be mandatory for police,d2,"This meme shows that the police is trustworthy, so no body camerass required."
6,4,Argument Pro Should body cameras be mandatory for police,d1,This meme shows that the police should get body cameras
7,4,Argument Pro Should body cameras be mandatory for police,d2,"This meme shows that the police is trustworthy, so no body camerass required."


In [ ]:
pt.text.scorer(wmodel="DPH")(docs_df).sort_values('score', ascending=False)

21:29:54.579 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


,qid,docno,body,text,rank,score,query
0,1,d1,this is the first document of many documents,this is the first document of many documents,0,0.674059,first
1,1,d2,this is another document,this is another document,1,0.000000,first
2,1,d3,the document of this document is unknown,the document of this document is unknown,2,0.000000,first
3,1,d4,the topic of this document is unknown,the topic of this document is unknown,3,0.000000,first
4,1,d66,document document,document document,4,0.000000,first


In [8]:
from pyterrier_t5 import MonoT5ReRanker

monoT5_base = MonoT5ReRanker()

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [9]:
monoT5_base(docs_df)

monoT5:   0%|          | 0/2 [00:00<?, ?batches/s]

,qid,query,docno,text,score,rank
0,1,Argument Against Teachers should get Tenure,d1,Teachers should not get tenure because they should stay motivated,-0.004496,0
1,1,Argument Against Teachers should get Tenure,d2,Teachers should get tenure to keep them motivated,-0.010931,1
2,2,Argument Pro Teachers should get Tenure,d1,Teachers should not get tenure because they should stay motivated,-0.038888,1
3,2,Argument Pro for Teachers should get Tenure,d2,Teachers should get tenure to keep them motivated,-0.006592,0
4,3,Argument Against Should body cameras be mandatory for police,d1,This meme shows that the police should get body cameras,-0.137567,1
5,3,Argument Against Should body cameras be mandatory for police,d2,"This meme shows that the police is trustworthy, so no body camerass required.",-0.104599,0
6,4,Argument Pro Should body cameras be mandatory for police,d1,This meme shows that the police should get body cameras,-0.104591,0
7,4,Argument Pro Should body cameras be mandatory for police,d2,"This meme shows that the police is trustworthy, so no body camerass required.",-0.186071,1


In [11]:
def _read_topics_trecxml(filename, tags=["title"], tokenise=True):
    """
    Parse a file containing topics in TREC-like XML format

    Args:
        filename(str): The path to the topics file

    Returns:
        pandas.Dataframe with columns=['qid','query']
    """
    import xml.etree.ElementTree as ET
    import pandas as pd
    tags=set(tags)
    topics=[]
    tree = ET.parse(filename)
    root = tree.getroot()
    from jnius import autoclass
    tokeniser = autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()
    for child in root.iter('topic'):
        qid = None
        query = ""
        for tag in child:
            if tag.tag == 'number':
              qid = tag.text
            if tag.tag in tags:
                query_text = tag.text
                if tokenise:
                    query_text = " ".join(tokeniser.getTokens(query_text))
                query += " " + query_text
        topics.append((str(qid), query))
    return pd.DataFrame(topics, columns=["qid", "query"])

def load_data(data_type=None):
    import json
    from tqdm import tqdm
    if data_type not in ['PRO', 'CON', 'ONTOPIC']:
      raise ValueError('X')

    topics = _read_topics_trecxml('task3-topics.xml')


    qrels = pd.read_csv('task3-qrels.txt', sep='\s', names=['qid', 'stance', 'docno', 'label'])
    qrels = qrels[qrels['stance'] == data_type]
    qrels['qid'] = qrels['qid'].astype(str)

    if data_type == 'PRO':
        topics['query'] = 'Argument Pro ' + topics['query']
    if data_type == 'CON':
        topics['query'] = 'Argument Against ' + topics['query']

    df_ret = []

    qid_to_query = {i['qid']:i['query'] for _, i in topics.iterrows()}
    did_to_text = {}
    for i in open('image_text_ocr.jsonl', 'r'):
      i = json.loads(i)
      did_to_text[i['image_id']] = ' '.join(i['html_text'])


    for _, i in qrels.iterrows():
      if str(i['docno']) not in did_to_text:
        continue
      df_ret += [{
          'qid': str(i['qid']),
          'query': qid_to_query[str(i['qid'])],
          'docno': i['docno'],
          'body': did_to_text[str(i['docno'])],
          'text': did_to_text[str(i['docno'])]
      }]

    return topics, qrels, pd.DataFrame(df_ret)

topics, qrels, df_docs = load_data('PRO')


In [12]:
df_docs.head(2)[['query', 'text']]

,query,text
0,Argument Pro should teachers get tenure,tide turn automatic tenure instead eliminate meaningful improve develop use measure teacher effectiveness issue hotly debate teacher policy tenure...
1,Argument Pro should teachers get tenure,source doctorate recipients u.s. universities national center science engineering statistics 2014 p. 9 \n


In [ ]:

topics, qrels, df_docs = load_data('PRO')

pt.io.write_results(monoT5_base(df_docs), 'monot5-base-pro-run.txt')
pt.io.write_results(pt.text.scorer(wmodel="DPH")(df_docs), 'dph-base-pro-run.txt')
pt.io.write_results(pt.text.scorer(wmodel="BM25")(df_docs), 'bm25-base-pro-run.txt')

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


monoT5:   0%|          | 0/1650 [00:00<?, ?batches/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1396 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
topics, qrels, df_docs = load_data('CON')


pt.io.write_results(monoT5_base(df_docs), 'monot5-base-con-run.txt')
pt.io.write_results(pt.text.scorer(wmodel="DPH")(df_docs), 'dph-base-con-run.txt')
pt.io.write_results(pt.text.scorer(wmodel="BM25")(df_docs), 'bm25-base-con-run.txt')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
3341it [00:00, 16792.78it/s]

skip
skip


6607it [00:00, 16051.63it/s]


skip
skip
skip
skip
skip
23:39:12.940 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
23:40:11.378 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


In [ ]:
def load_run(run_name):
  ret = pt.io.read_results(run_name)
  ret['qid'] = ret['qid'].astype(str)
  
  return pt.transformer.Transformer.from_df(ret)

In [ ]:
topics, qrels, df_docs = load_data('PRO')

pt.Experiment(
    [load_run('dph-base-pro-run.txt'), load_run('bm25-base-pro-run.txt'), load_run('monot5-base-pro-run.txt')],
    topics,
    qrels,
    eval_metrics=["P_5", "P_10"],
    names=['DPH', 'BM25', 'MonoT5'],
    baseline=0,
    correction='bonferroni'
)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)
3271it [00:00, 16450.79it/s]

skip
skip


6607it [00:00, 15969.32it/s]


skip
skip
skip
skip
skip


,name,P_5,P_10,P_5 +,P_5 -,P_5 p-value,P_5 reject,P_5 p-value corrected,P_10 +,P_10 -,P_10 p-value,P_10 reject,P_10 p-value corrected
0,DPH,0.584,0.584,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,False,NaN
1,BM25,0.560,0.546,9.0,14.0,0.293435,False,0.880305,10.0,23.0,0.045238,False,0.135713
2,MonoT5,0.552,0.584,10.0,17.0,0.220486,False,0.661457,18.0,15.0,1.000000,False,1.000000


In [ ]:

topics, qrels, df_docs = load_data('CON')

pt.Experiment(
    [load_run('dph-base-con-run.txt'), load_run('bm25-base-con-run.txt'), load_run('monot5-base-pro-run.txt')],
    topics,
    qrels,
    eval_metrics=["P_5", "P_10"],
    names=['DPH', 'BM25', 'MonoT5'],
)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)
3162it [00:00, 15747.13it/s]

skip
skip


6607it [00:00, 15698.71it/s]


skip
skip
skip
skip
skip


,name,P_5,P_10
0,DPH,0.200,0.192
1,BM25,0.212,0.186
2,MonoT5,0.184,0.180


In [ ]:
topics, qrels, df_docs = load_data('ONTOPIC')

pt.Experiment(
    [load_run('dph-base-pro-run.txt'), load_run('bm25-base-pro-run.txt'), load_run('monot5-base-pro-run.txt')],
    topics,
    qrels,
    eval_metrics=["P_5", "P_10"],
    names=['DPH', 'BM25', 'MonoT5'],
    correction='bonferroni',
    baseline=0
)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)
3226it [00:00, 16132.07it/s]

skip
skip


6607it [00:00, 16026.63it/s]


skip
skip
skip
skip
skip


,name,P_5,P_10,P_5 +,P_5 -,P_5 p-value,P_5 reject,P_5 p-value corrected,P_10 +,P_10 -,P_10 p-value,P_10 reject,P_10 p-value corrected
0,DPH,0.828,0.826,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,False,NaN
1,BM25,0.796,0.766,7.0,15.0,0.131931,False,0.395794,4.0,23.0,0.000360,True,0.001081
2,MonoT5,0.800,0.806,6.0,16.0,0.226574,False,0.679723,14.0,21.0,0.242433,False,0.727298
